# Data cleaning and Preprocessing

**Read Original Data**

In [112]:
with open("original_data.txt","r",encoding="utf-8") as fp:
    data = fp.read().split("\n")


**Remove Irrelevant colums**

In [113]:
def process_colums(string):
    selected_col = [1,2,3,5,6,9,15,17,19,]
    if len(string) < 10:
        return None
    row_list = string.split("|")
    new_row = [row_list[i] for i in selected_col]
    return new_row

cleaned_data=[]
for row in data:
    res = process_colums(row)
    if res:
        cleaned_data.append(res)

print(cleaned_data[1])
    
    
    

['176', '150', '100', '北京', '2000人以上', '3', '3', '行政实习生（物业管理）', '互联网/游戏/软件']


**Compute keyword list for job names**

In [77]:
import jieba

word_dict = {}
job_name_list = []
for row in data:
    if len(row) > 5:
        name = row.split("|")[17]
        job_name_list.append(name)
        for word in jieba.cut(name):
            if len(word) > 1:
                if word in word_dict:
                    word_dict[word] +=1

                else:
                    word_dict[word] = 1
ignore_list = ["实习生","实习","方向","2021","前端开发","视频剪辑"]
for ignore_label in ignore_list:
    word_dict.pop(ignore_label)

l = sorted(word_dict.items(), key=lambda x: x[1])
word_list = []
for item in l:
    if item[1] >= 40:
        word_list.append(item[0])
count = 0
for name in job_name_list:
    for word in word_list:
        if word in name:
            count += 1
            break
print("data covered by the word list:",count)
print(word_list)

data covered by the word list: 4168
['Java', 'HRBP', '文案', '平台', '实施', '前端', '管理', '社群', '培训', '校招', '人事', 'UI', '活动', '研发', '顾问', '设计师', '视觉', '用户', '项目', '行政', '管培生', '海外', '市场', '运维', '数据', 'HR', '策划', '电商', '商务', '人力资源', '客服', '编辑', '测试', '游戏', '算法', '视频', '设计', '招聘', '媒体', '内容', '专员', '产品', '销售', '开发', '工程师', '助理', '运营']


In [114]:
scale_vector = ["2000人以上",("500-2000人","500—2000人"),("150-500人","50-100人","50-150人","50-200人"),"15-50人","少于50人","未知规模"]
job_vector = ['Java', 'HRBP', '文案', '平台', '实施', '前端', '管理', '社群', '培训', '校招', '人事', 'UI', '活动', '研发', '顾问', '设计师', '视觉', '用户', '项目', '行政', '管培生', '海外', '市场', '运维', '数据', 'HR', '策划', '电商', '商务', '人力资源', '客服', '编辑', '测试', '游戏', '算法', '视频', '设计', '招聘', '媒体', '内容', '专员', '产品', '销售', '开发', '工程师', '助理', '运营']
industry_vector = ['互联网','教育','房产','广告','金融','企业服务','快消','电子','医疗','汽车']

**Map string into digits, texts into vectors**

In [115]:
final_data = []
def map(string, vector):
    res = ["0"]*len(vector)
    for i in range(len(vector)):
        if isinstance(vector[i],tuple):
            for sub_label in vector[i]:
                if sub_label in string:
                    res[i] = "1"
        elif vector[i] in string:
            res[i] = "1"
    return res


for row in cleaned_data:
    try:
        row += map(row[4],scale_vector)
        row += map(row[8],industry_vector)
        row += map(row[7],job_vector)
        row.pop(4)
        row.pop(6)
        row.pop(6)
        final_data.append(row)
    except:
        pass
    
    

    

    
print(final_data[1])

['176', '150', '100', '北京', '3', '3', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


In [116]:
string = ""
for row in final_data:
    string += ",".join(row)+"\n"

header = "intern_num,max_sal,min_sal,city,month_num,day,2000人以上,500-2000人,50-500人,50-100人,15-50人,少于50人,未知规模"
header += ",".join(industry_vector)
header += ",".join(job_vector)
string = header + "\n"+ string
with open("formated_data.csv","w") as fp:
    fp.write(string)
    